In [ ]:
# OpenSoundscape imports
from opensoundscape import Audio, Spectrogram
from opensoundscape.annotations import BoxedAnnotations

# General-purpose packages
import numpy as np
import pandas as pd
from glob import glob
from pathlib import Path

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for big visuals
%config InlineBackend.figure_format = 'retina'

Loading Raven Files

In [ ]:
from glob import glob
import os
import re

# Assuming your current working directory is the base path
base_path = os.getcwd()

# Folder name
folder_name = "ravenSelectionTables"

# Construct the relative path
relative_folder_path = os.path.join(base_path, folder_name)

# Use glob to find all .txt files directly in the folder
selections = glob(os.path.join(relative_folder_path, "*.txt"))
# Regular expression pattern to extract the date and time part
pattern = r"(\d{8}_\d{6})" #NOTE THIS MAY NOT WORK FOR ALL RECORDINGS IN OTHER FILES

# Iterate through files and rename to include only the date and time part
for old_file_path in selections:
    old_file_name = os.path.basename(old_file_path)
    
    # Use regex to find the date and time part
    match = re.search(pattern, old_file_name)
    
    if match:
        new_file_name = match.group(1)
        
        # Construct the new file path
        new_file_path = os.path.join(relative_folder_path, new_file_name + ".txt")
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {old_file_path} to {new_file_path}")
    else:
        print(f"No match found in filename: {old_file_name}")

# Optional: Print the updated list of files in ravenSelectionTables
updated_selections = glob(os.path.join(relative_folder_path, "*.txt"))
print(updated_selections)




Renamed: c:\Users\jonat\OneDrive\Documents\bobwhite\ravenSelectionTables\20221101_060000.txt to c:\Users\jonat\OneDrive\Documents\bobwhite\ravenSelectionTables\20221101_060000.txt
['c:\\Users\\jonat\\OneDrive\\Documents\\bobwhite\\ravenSelectionTables\\20221101_060000.txt']


In [ ]:
# create a list of audio files, one corresponding to each Raven file
'''so this is going through the ravenSelectionTables folder and finidng the audio files. 
For now, we manually make sure the audio name is EXACT SAME as the txt file name.'''
audio_files = glob(os.path.join(relative_folder_path, "*.wav"))

audio_files


['c:\\Users\\jonat\\OneDrive\\Documents\\bobwhite\\ravenSelectionTables\\20221101_060000.wav']

It seems like the column "annotations" is a number, while at the end there is a column "Annotations" thats not supposed to be there

In [ ]:
from opensoundscape import BoxedAnnotations

all_annotations = BoxedAnnotations.from_raven_files(
    updated_selections,audio_files)

#RENAME DUPLICATE ANNOTATION TEST
all_annotations.df.rename(columns={'annotation': 'length'}, inplace=True)
all_annotations.df.rename(columns={'Annotation': 'annotation'}, inplace=True)

all_annotations.df.head(10)

,audio_file,annotation_file,length,start_time,end_time,low_f,high_f,Channel,View,Delta Freq (Hz),Avg Power Density (dB FS/Hz),Selection,annotation
0,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.2915,2.943193,3.234736,1757.469,2636.204,1,Spectrogram 1,878.735,-73.57,1,NOBO?
1,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.4304,4.234311,4.664683,1675.088,2773.506,1,Spectrogram 1,1098.418,-74.02,2,NOBO?
2,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.4720,32.749963,33.221985,1586.942,2752.858,1,Spectrogram 1,1165.917,-73.56,5,NOBO
3,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.3471,33.485762,33.832836,1716.488,3011.951,1,Spectrogram 1,1295.463,-72.28,6,NOBO
4,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.2221,34.485337,34.707465,1554.555,2688.085,1,Spectrogram 1,1133.530,-70.64,7,NOBO
5,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.3332,34.665816,34.999007,1198.303,3303.430,1,Spectrogram 1,2105.127,-73.37,8,NOBO
6,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.2499,35.012890,35.262784,1716.488,3011.951,1,Spectrogram 1,1295.463,-73.57,9,NOBO
7,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.3471,35.804221,36.151295,1392.623,2040.354,1,Spectrogram 1,647.731,-73.93,10,NOBO
8,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.3332,37.345232,37.678424,1327.849,2072.740,1,Spectrogram 1,744.891,-71.08,11,NOBO
9,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,0.3471,38.705765,39.052839,1263.076,2040.354,1,Spectrogram 1,777.278,-72.93,12,NOBO


In [ ]:
all_annotations.df.annotation.value_counts() 

annotation
NOBO     151
NOBO?      6
Name: count, dtype: int64

In [ ]:
#keep only annotations that say "NOBO"
class_list = ['NOBO']
thrush_annotations = all_annotations.subset(class_list)
thrush_annotations.df.head(5)

,audio_file,annotation_file,annotation,start_time,end_time,low_f,high_f,Channel,View,Delta Freq (Hz),Avg Power Density (dB FS/Hz),length,Selection
2,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,NOBO,32.749963,33.221985,1586.942,2752.858,1,Spectrogram 1,1165.917,-73.56,0.4720,5
3,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,NOBO,33.485762,33.832836,1716.488,3011.951,1,Spectrogram 1,1295.463,-72.28,0.3471,6
4,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,NOBO,34.485337,34.707465,1554.555,2688.085,1,Spectrogram 1,1133.530,-70.64,0.2221,7
5,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,NOBO,34.665816,34.999007,1198.303,3303.430,1,Spectrogram 1,2105.127,-73.37,0.3332,8
6,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,c:\Users\jonat\OneDrive\Documents\bobwhite\rav...,NOBO,35.012890,35.262784,1716.488,3011.951,1,Spectrogram 1,1295.463,-73.57,0.2499,9


Now time for one hot encoding for the model


In [ ]:
# create labels for fixed-duration (2 second) clips 

'''since we aren't in the ravenselectiontable directory since the beginning, 
I ASSUME that i need to reference the audio file path manually under audio_files parameter'''

audio_location = glob(os.path.join(relative_folder_path, "*.wav"))

print(audio_location)

labels = all_annotations.one_hot_clip_labels(
  clip_duration=3,
  clip_overlap=1,
  min_label_overlap=0.1,
  class_subset=class_list, 
  audio_files= audio_location
)

# Set the option to display all rows
pd.set_option('display.max_rows', None)

print(labels)

['c:\\Users\\jonat\\OneDrive\\Documents\\bobwhite\\ravenSelectionTables\\20221101_060000.wav']
                                                                        NOBO
file                                               start_time end_time      
c:\Users\jonat\OneDrive\Documents\bobwhite\rave... 0.0        3.0        0.0
                                                   2.0        5.0        0.0
                                                   4.0        7.0        0.0
                                                   6.0        9.0        0.0
                                                   8.0        11.0       0.0
                                                   10.0       13.0       0.0
                                                   12.0       15.0       0.0
                                                   14.0       17.0       0.0
                                                   16.0       19.0       0.0
                                                   18.0   

Training the model

In [ ]:
from sklearn.model_selection import train_test_split
from opensoundscape import CNN
from opensoundscape.ml.cnn import load_model

# split the labels into training and validation sets
train_df, validation_df = train_test_split(labels, test_size=0.2, random_state=1)


# Define constants
SAMPLE_DURATION = 4.0
NUM_EPOCHS = 10
BATCH_SIZE = 256
SAVE_INTERVAL = 1000

# Specify paths
model_path = './bird_training/bobwhite/best.model'
save_path = './bird_training/bobwhite'

# Create model object
classes = train_df.columns #in this case, there's just one class: ["woodcock"]

if os.path.exists(model_path):
    model = load_model(model_path)
else:
    model = CNN('resnet18', classes=class_list, sample_duration=SAMPLE_DURATION)

    model.train(
        train_df=train_df,
        validation_df=validation_df,
        save_path=save_path,
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        save_interval=SAVE_INTERVAL,
        num_workers=0, #specify 4 if you have 4 CPU processes, eg; 0 means only the root process
    )

    model.save(model_path)
    model = load_model(model_path)


Training Epoch 0


  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\jonat\anaconda3\envs\bird\lib\site-packages\opensoundscape\sample.py:152: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.Tensor([s.labels for s in samples]),


Epoch: 0 [batch 0/1, 0.00%] 
	DistLoss: 0.686
Metrics:
Metrics:
	MAP: 0.356

Validation.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\jonat\anaconda3\envs\bird\lib\site-packages\opensoundscape\sample.py:152: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "labels": torch.Tensor([s.labels for s in samples]),


Metrics:
	MAP: 0.357

Training Epoch 2


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 